In [19]:
# Bokeh Libraries
from bokeh.io import output_file
from bokeh.plotting import figure, show
from bokeh.models import HoverTool
import pandas as pd

# The figure will be rendered inline in my Jupyter Notebook
output_file('analysis2.html', title='a2')

# read file into Dataframe
file_path = 'combined_sales_file.csv'
df_table = pd.read_csv(file_path)

# create a new dataframe
monthly_sku_amount = pd.DataFrame(columns=['month', 'premium', 'unlockcharactermanager'])

# change the date form
df = pd.DataFrame(df_table)
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], format='mixed', errors='coerce')
df['month'] = df['Transaction Date'].dt.month

# groupby premuium and unlockcharactermanager by month
premium_amount = df[df['Sku Id'] == 'premium'].groupby('month')['Amount'].sum()
unlockcharactermanager_amount = df[df['Sku Id'] == 'unlockcharactermanager'].groupby('month')['Amount'].sum()

# use month as a index
monthly_sku_amount['month'] = premium_amount.index
# add line premium and unlockcharactermanager
monthly_sku_amount['premium'] = premium_amount.values
monthly_sku_amount['unlockcharactermanager'] = unlockcharactermanager_amount.values

print(monthly_sku_amount)

# add HoverTool
hover = HoverTool(
    tooltips=[("Month", "@month"),
              ("Premium", "@premium{0,0}"), 
              ("UnlockCharacterManager", "@unlockcharactermanager{0,0}")],  
    mode='vline'  
)

# use bar chart to visualize
p = figure(width=550, 
           height=600, 
           title='Monthly Transaction Analysis', 
           x_axis_label='Month', 
           y_axis_label = 'Transaction Amount',
           tools=[hover])

p.vbar(x=monthly_sku_amount['month'], 
       top=monthly_sku_amount['premium'], 
       width=0.3, 
       color='orange', 
       legend_label='premium')
p.vbar(x=monthly_sku_amount['month']+0.3, 
       top=monthly_sku_amount['unlockcharactermanager'], 
       width=0.3, 
       color='blue', 
       legend_label='unlockcharactermanager')

p.legend.click_policy = "hide"
show(p)

   month  premium  unlockcharactermanager
0      6   224.72                  436.94
1      7   321.91                  671.75
2      8   332.53                  586.09
3      9   226.71                  547.75
4     10   216.28                  517.68
5     11   336.86                  734.29
6     12   365.07                  701.01
